# Первый подход

Попытка решить задачу через небольшой набор фич и катбуст.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
from catboost import CatBoostClassifier, CatBoostRegressor

In [3]:
from settings import *
from load import *
from score import *
from a1 import get_data

In [4]:
TEST_SIZE = 0.2

In [5]:
%%time
df0 = get_data()
target = load_target()

df0 = df0.loc[target.index]
# df0 = df0.iloc[:100000]
target = target.loc[df0.index]
df0 = df0.loc[target['user_id']]

target_sex = target['is_male'].dropna().astype(int)
target_age = target['age_bin'].dropna().astype(int)
target_age_cont = target['age'].dropna().astype('float32')
target_age = target_age[target_age != -1]
target_age = target_age.replace({x: x-1 for x in sorted(target_age.unique())})

df0.shape

D:\mts_cup\data\approach_1\f_region_category.pickle


100%|█████████████████████████████████████████████████████████████████████| 415316/415316 [02:02<00:00, 3382.00it/s]


CPU times: total: 2min 9s
Wall time: 2min 9s


(270000, 215)

In [6]:
df = df0
cat_features = list(df.dtypes[df.dtypes == 'category'].index)
cat_features += ['most_loaded_dow', 'less_loaded_dow']

noncat_features = ['region_count']
for col in [x for x in noncat_features if x in df]:
    df[col] = df[col].astype('float32')

cat_features = [x for x in cat_features if x not in noncat_features]
cat_features = [x for x in cat_features if x in df]

noncat_features = [x for x in df if x not in cat_features]
for col in cat_features:
    df[col] = df[col].astype(str)
d = dict()

In [7]:
%%time
f1, f2, t1, t2 = train_test_split(df.loc[target_sex.index], target_sex, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=target_sex)
model = CatBoostClassifier(cat_features=cat_features,
                           random_state=RANDOM_SEED,
                           learning_rate=1e-4,
                           verbose=0)
model.fit(f1, t1)
print('score', score_sex_model(model, f2, t2))
d['sex_1'] = (model, f1, t1)
d['sex_2'] = (model, f2, t2)

score 0.4971036731205345
CPU times: total: 14min 21s
Wall time: 1min 23s


In [8]:
f1, f2, t1, t2 = train_test_split(df.loc[target_age.index], target_age, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=target_age)
model = CatBoostClassifier(cat_features=cat_features,
                           auto_class_weights='Balanced',
                           loss_function='MultiClass',
                           # eval_metric='TotalF1:average=Weighted',
                           random_state=RANDOM_SEED,
                           learning_rate=1e-3,
                           verbose=0)
model.fit(f1, t1)
print('score', score_age_model(model, f2, t2))
d['age_1'] = (model, f1, t1)
d['age_2'] = (model, f2, t2)

score 0.2493217796023273


In [10]:
score_model(sex=d['sex_1'], age=d['age_1']), score_model(sex=d['sex_2'], age=d['age_2'])

(1.005482526374551, 0.9983555742286665)

In [11]:
y = pd.Series(np.reshape(model.predict(f1), (-1, )), index=t1.index)
classes = list(range(t1.max()))
print(classification_report(t1, y))

              precision    recall  f1-score   support

          -1       0.02      0.65      0.03       862
           0       0.28      0.42      0.34     26113
           1       0.44      0.23      0.30     69816
           2       0.39      0.16      0.23     61989
           3       0.25      0.18      0.21     33953
           4       0.20      0.22      0.21     18864
           5       0.07      0.56      0.12      4402

    accuracy                           0.23    215999
   macro avg       0.24      0.34      0.20    215999
weighted avg       0.35      0.23      0.26    215999

